# Introduction to the concept of retrieval augmented generation (RAG)

> *PLEASE NOTE: This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio. Also, you should have provisioned access in your AWS account for the required Bedrock models as described [here](https://catalog.us-east-1.prod.workshops.aws/event/dashboard/en-US/workshop/20-introduction/21-bedrock).*

---

Question Answering (QA) is an important task that involves extracting answers to factual queries posed in natural language. Typically, a QA system processes a query against a knowledge base containing structured or unstructured data and generates a response with accurate information. Ensuring high accuracy is key to developing a useful, reliable and trustworthy question answering system, especially for enterprise use cases. However, in this notebook, we will highlight a well-documented issue with LLMs: LLM's are unable to answer questions outside of their training data.

In [ ]:
import sys
import os
module_path = "../.."
sys.path.append(os.path.abspath(module_path))
from utils.environment_validation import validate_environment, validate_model_access
validate_environment()

In [ ]:
required_models = [
    "amazon.titan-embed-text-v1",
    "amazon.titan-embed-text-v2:0",
    "us.anthropic.claude-3-5-haiku-20241022-v1:0",
    "us.anthropic.claude-3-5-sonnet-20241022-v2:0",
]
validate_model_access(required_models)

---
## Setup the `boto3` client connection to Amazon Bedrock

Similar to notebook "01_workshop_setup.ipynb", we will create a client side connection to Amazon Bedrock using the `boto3` library.

In [ ]:
from IPython.display import Markdown, display

import json
from rich import print as rprint

import boto3
import botocore

from utils.prompt_utils import prompts_to_messages

boto3_bedrock = boto3.client("bedrock-runtime")

---
## Highlighting the Contextual Issue

To illustrate the problem that RAG helps address, let's first illustrate the issue with requesting factual information from a model. As an example, we'll ask the model to tell us "What is the current Federal Funds Rate as of February?". The Claude 3.5 Haiku model's training data cuts off in Q2 2024, nor does the model have a concept of time to interpret what "current" means. Therefore Claude will not be able to accurately answer this question. In some case the LLM may be aware of its limitations and provide a response along the lines of "I'm not sure" or "I don't know", however in many cases the model will provide an incorrect answer.

In [ ]:
import json
prompt = "What is the current Federal Funds Rate?"


body = json.dumps({
    "max_tokens": 500,
    "messages": prompts_to_messages(prompt),
    "anthropic_version": "bedrock-2023-05-31"
})


modelId = "us.anthropic.claude-3-5-haiku-20241022-v1:0"

accept = "application/json"
contentType = "application/json"

response = boto3_bedrock.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)

response_body = json.loads(response.get("body").read())

rprint(response_body.get("content")[0]["text"])

The answer provided by Claude would either be incorrect based on stale information or Claude may indicate that it does not have the requisite information to answer the question. 

---
## Manually Providing Correct Context

In order to have Claude correctly answer the question provided, we need to provide the model context which is relevant to the question. Below example provides additional context via Federal Reserve's FOMC statement.

We can inject this context into the prompt as shown below and ask the LLM to answer our question based on the context provided.

In [ ]:
prompt = '''Answer question provided below by using the context provided. Do not use any information other than what is provided in the context. If the context is insufficient, please respond with "Insufficient information".

<context>
Recent indicators suggest that economic activity has continued to expand at a solid pace. Since earlier in the year, labor market conditions have generally eased, and the unemployment rate has moved up but remains low. Inflation has made progress toward the Committee's 2 percent objective but remains somewhat elevated.

The Committee seeks to achieve maximum employment and inflation at the rate of 2 percent over the longer run. The Committee judges that the risks to achieving its employment and inflation goals are roughly in balance. The economic outlook is uncertain, and the Committee is attentive to the risks to both sides of its dual mandate.

In support of its goals, the Committee decided to lower the target range for the federal funds rate by 1/4 percentage point to 4-1/4 to 4-1/2 percent. In considering the extent and timing of additional adjustments to the target range for the federal funds rate, the Committee will carefully assess incoming data, the evolving outlook, and the balance of risks. The Committee will continue reducing its holdings of Treasury securities and agency debt and agency mortgage‑backed securities. The Committee is strongly committed to supporting maximum employment and returning inflation to its 2 percent objective.

In assessing the appropriate stance of monetary policy, the Committee will continue to monitor the implications of incoming information for the economic outlook. The Committee would be prepared to adjust the stance of monetary policy as appropriate if risks emerge that could impede the attainment of the Committee's goals. The Committee's assessments will take into account a wide range of information, including readings on labor market conditions, inflation pressures and inflation expectations, and financial and international developments.
</context>

Question: What is the current Federal Funds Rate?

'''

body = json.dumps({
    "max_tokens": 256,
    "messages": prompts_to_messages(prompt),
    "anthropic_version": "bedrock-2023-05-31"
})


response = boto3_bedrock.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())
rprint(response_body.get("content")[0]["text"])

Now you can see that the model answers the question accurately based on the factual context. However, this context had to be added manually to the prompt. In a production setting, we need a way to automate the retrieval of this information.

## Providing External Context Automatically
In practice, a RAG solution would dynamically provide the relevant context to the LLM. This is done by performing a search over a large corpus of documents to find the most relevant information to the question. Then providing the relevant context to the LLM along with the question. This is a powerful technique that allows the LLM to answer questions that are not in its training data.

In subsequent sections, you will learn how to build your own search engine, but here will illustrate the RAG concept using Wikipedia search. Wikipedia is a commonly used data source for training LLMs, so we will ask a question about a recent event that would not be in the training data. 

In [ ]:
%pip install wikipedia
%pip install wikipedia-api

In [ ]:
import wikipedia
import wikipediaapi

wiki_wiki = wikipediaapi.Wikipedia('RAGexample','en')

query = "What is the Federal Funds Rate as of February 2025?"

search_results = wikipedia.search(query)
page_content = wiki_wiki.page(search_results[0]).text

prompt = f'''Use the context provided to answer the question below. If the context is insufficient, please respond with "Insufficient information".

<context>
{page_content}
</context>

Question: {query}
'''

body = json.dumps({
    "max_tokens": 256,
    "temperature": 0.2,
    "messages": prompts_to_messages(prompt),
    "anthropic_version": "bedrock-2023-05-31"
})



response = boto3_bedrock.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())

rprint(response_body.get("content")[0]["text"])

---
## Quick Note: Long Context Windows

One known limitation for RAG based solutions is the need for inclusion of lots of text into a prompt for an LLM. Fortunately, Claude can help this issue by providing an input token limit of 200k tokens. This limit [corresponds to around 150k words](https://www.anthropic.com/news/claude-2-1) which is an astounding amount of text.

Let's take a look at an example of Claude handling this large context size...

In [ ]:
book = ''
with open('../data/book/book.txt', 'r') as f:
    book = f.read()
print('Context:', book[0:53], '...')
print('The context contains', len(book.split(' ')), 'words')

In [ ]:
prompt =f'''

Summarize the plot of this book.

<book>
{book}
</book>

'''

body = json.dumps({
    "max_tokens": 1000,
    "messages": prompts_to_messages(prompt),
    "anthropic_version": "bedrock-2023-05-31"
})



response = boto3_bedrock.invoke_model(
    body=body, modelId=modelId, accept='application/json', contentType='application/json'
)
response_body = json.loads(response.get('body').read())
rprint(response_body.get("content")[0]["text"])

---
## Next steps

Now you have been able to see a concrete example where LLMs can be improved with correct context injected into a prompt, lets move on to the next notebook to see how we can automate this process using OpenSearch vector database.